### Créer la matrice d'adjacence:

Avec un masque et des valeurs de tenseurs de diffusion

In [1]:
!python build_adj_matrix.py ../data/wm.nii.gz ../data/fodf.nii.gz graph

### Afficher le graphe: 

In [10]:
from IPython.display import Image, display  # type: ignore
!python draw_adj_matrix.py graph.npz
display(Image(filename='graph_adj_mat.png'))


Traceback (most recent call last):
  File "c:\Users\harsh\quactography\scripts\draw_adj_matrix.py", line 52, in <module>
    main()
  File "c:\Users\harsh\quactography\scripts\draw_adj_matrix.py", line 24, in main
    weighted_graph, node_indices, vol_dim = load_graph(args.in_graph)
  File "c:\users\harsh\quactography\quactography\adj_matrix\io.py", line 12, in load_graph
    npzfile = np.load(in_file)
  File "c:\Users\harsh\.virtualenv\quactography\lib\site-packages\numpy\lib\npyio.py", line 405, in load
    fid = stack.enter_context(open(os_fspath(file), "rb"))
FileNotFoundError: [Errno 2] No such file or directory: 'graph.npz'


### Filtrer le graphe pour avoir uniquement les noeuds fortement connectés:

In [3]:
!python build_adj_matrix.py ../data/wm.nii.gz ../data/fodf.nii.gz graph --threshold 0.4


In [11]:
!python draw_adj_matrix.py graph.npz
display(Image(filename='graph_adj_mat.png'))

Traceback (most recent call last):
  File "c:\Users\harsh\quactography\scripts\draw_adj_matrix.py", line 52, in <module>
    main()
  File "c:\Users\harsh\quactography\scripts\draw_adj_matrix.py", line 24, in main
    weighted_graph, node_indices, vol_dim = load_graph(args.in_graph)
  File "c:\users\harsh\quactography\quactography\adj_matrix\io.py", line 12, in load_graph
    npzfile = np.load(in_file)
  File "c:\Users\harsh\.virtualenv\quactography\lib\site-packages\numpy\lib\npyio.py", line 405, in load
    fid = stack.enter_context(open(os_fspath(file), "rb"))
FileNotFoundError: [Errno 2] No such file or directory: 'graph.npz'


### Créer une matrice d'adjacence aléatoire:

In [5]:
!python build_random_adj_matrix.py 5 6 True rand_graph

4
6


In [12]:
!python draw_random_adj_matrix.py rand_graph rand_graph_visu
display(Image(filename='rand_graph_visu.png'))

Traceback (most recent call last):
  File "c:\Users\harsh\quactography\scripts\draw_random_adj_matrix.py", line 49, in <module>
    main()
  File "c:\Users\harsh\quactography\scripts\draw_random_adj_matrix.py", line 29, in main
    weighted_graph, node_indices, _ = load_graph(args.in_graph + ".npz")
  File "c:\users\harsh\quactography\quactography\adj_matrix\io.py", line 12, in load_graph
    npzfile = np.load(in_file)
  File "c:\Users\harsh\.virtualenv\quactography\lib\site-packages\numpy\lib\npyio.py", line 405, in load
    fid = stack.enter_context(open(os_fspath(file), "rb"))
FileNotFoundError: [Errno 2] No such file or directory: 'rand_graph.npz'


### Lancer l'algorithme de qaoa pour maximiser la valeur absolue des poids: 

nom de la matrice d'adjacence, noeud départ, noeud fin, nom du fichier d'infos, valeurs de la constante de pénalité, nombre de répétitions de QAOA, nombre de processeurs 

In [7]:
!python find_max_intensity_diffusion_path.py rand_graph 1 0 qaoa_solver_infos --alphas 1 2 3  --reps 1 --multiprocess -npr 2

------------------------MULTIPROCESS SOLVER FINISHED-------------------------------


### Visualiser les chemins qui maximise la valeur absolue des poids:

In [13]:
!python plot_distribution_probabilities.py qaoa_solver_infos_alpha_1.npz qaoa_solver_infos_alpha_2.npz qaoa_solver_infos_alpha_3.npz visu_total_dist visu_selected_dist
display(Image(filename='visu_selected_dist_0.png'))
display(Image(filename=r'..\output\img\visu_tout_alpha_4.png'))

Traceback (most recent call last):
  File "c:\Users\harsh\quactography\scripts\plot_distribution_probabilities.py", line 40, in <module>
    main()
  File "c:\Users\harsh\quactography\scripts\plot_distribution_probabilities.py", line 32, in main
    plot_distribution_of_probabilities(
  File "c:\users\harsh\quactography\quactography\visu\dist_prob.py", line 12, in plot_distribution_of_probabilities
    _, dist_binary_prob, min_cost, h, _, _ = load_optimization_results(in_file)
  File "c:\users\harsh\quactography\quactography\solver\io.py", line 19, in load_optimization_results
    data = np.load(in_file, allow_pickle=True)
  File "c:\Users\harsh\.virtualenv\quactography\lib\site-packages\numpy\lib\npyio.py", line 405, in load
    fid = stack.enter_context(open(os_fspath(file), "rb"))
FileNotFoundError: [Errno 2] No such file or directory: 'qaoa_solver_infos_alpha_1.npz'


### Visualiser les chemins optimaux:

In [14]:
!python plot_optimal_paths.py graph qaoa_solver_infos_alpha_1.npz qaoa_solver_infos_alpha_2.npz qaoa_solver_infos_alpha_3.npz opt_paths
display(Image(filename='opt_paths_0_alpha_1.00.png'))
display(Image(filename='opt_paths_1_alpha_2.00.png'))
display(Image(filename='opt_paths_2_alpha_3.00.png'))

Traceback (most recent call last):
  File "c:\Users\harsh\quactography\scripts\plot_optimal_paths.py", line 36, in <module>
    main()
  File "c:\Users\harsh\quactography\scripts\plot_optimal_paths.py", line 28, in main
    visualize_optimal_paths(
  File "c:\users\harsh\quactography\quactography\visu\optimal_paths.py", line 26, in visualize_optimal_paths
    _, _, min_cost, h, bin_str, reps = load_optimization_results(in_file)
  File "c:\users\harsh\quactography\quactography\solver\io.py", line 19, in load_optimization_results
    data = np.load(in_file, allow_pickle=True)
  File "c:\Users\harsh\.virtualenv\quactography\lib\site-packages\numpy\lib\npyio.py", line 405, in load
    fid = stack.enter_context(open(os_fspath(file), "rb"))
FileNotFoundError: [Errno 2] No such file or directory: 'qaoa_solver_infos_alpha_1.npz'
